In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
import trimesh
import numpy as np

class ModelNet40Dataset(Dataset):
    def __init__(self, root_dir, transform=None, voxel_size=0.05):
        """
        Inizializza il dataset con il percorso delle mesh 3D e il parametro di voxelizzazione.
        
        :param root_dir: Cartella contenente i file .off
        :param transform: Trasformazioni da applicare ai dati
        :param voxel_size: Dimensione dei voxel nella voxelizzazione
        """
        self.root_dir = root_dir
        self.transform = transform
        self.files = [os.path.join(root, name) 
                      for root, dirs, files in os.walk(root_dir) 
                      for name in files if name.endswith(".off")]
        
        print(f"Found {len(self.files)} .off files in {root_dir}.")
    
    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        # Ottieni il percorso del file .off
        file_path = self.files[idx]
        
        # Carica la mesh 3D utilizzando trimesh
        mesh = trimesh.load(file_path)
        
        # Voxelizza la mesh 3D con una dimensione del voxel specificata
        voxelized_mesh = mesh.voxelized(pitch=0.05)  # pitch è la dimensione del voxel
        
        # Ottieni la griglia di voxel (numpy array)
        voxel_grid = voxelized_mesh.matrix
        
        # Converti in un tensore PyTorch
        voxel_tensor = torch.tensor(voxel_grid, dtype=torch.float32)
        
        # Se il modello è vuoto (griglia piena di zeri), potresti voler aggiungere una condizione
        # per gestire i casi in cui il modello è vuoto.
        
        if self.transform:
            voxel_tensor = self.transform(voxel_tensor)
        
        return voxel_tensor

# Path alla cartella del dataset ModelNet40
dataset_path = 'path/to/ModelNet40'  # Sostituisci con il tuo percorso

# Crea un'istanza del dataset
modelnet_dataset = ModelNet40Dataset(root_dir=dataset_path)

# Crea il DataLoader per batching e mescolamento
dataloader = DataLoader(modelnet_dataset, batch_size=8, shuffle=True)

# Testa il DataLoader
for batch in dataloader:
    print("Batch shape:", batch.shape)  # La forma attesa dovrebbe essere [batch_size, depth, height, width]
    break